# GPU Configuration

In [1]:
!nvidia-smi

Sat Feb 25 06:32:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Importing Packages

In [4]:
# Common Imports
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from time import time
from google.colab import drive
from matplotlib.image import imread
from sklearn.model_selection import KFold

# PyTorch Imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms

from torchsummary import summary as Summary
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler

# Classes

## Dataset Class

In [5]:
class Medical_Dataset(Dataset):

    def __init__(self, root_folder, class_folders=None, XTransformer=None, YTransformer=None, 
                 Validation_Type="Split", Train_Dataset=True, Validation_Subjects=None):
        
        self.XTransformer = XTransformer
        self.YTransformer = YTransformer
        self.Dataset_Size = 0
         
        if not class_folders:
    
            class_folders = os.listdir(root_folder)
            class_folders.sort()

        #################### Split and KFold Validation ####################
        if Validation_Type == "Split" or Validation_Type == "KFold" :
            
            for class_folder in class_folders:

                class_folder_path = os.path.join(root_folder, class_folder)
                self.Dataset_Size += len(os.listdir(class_folder_path))

            Total_Images = np.empty(0)
            Total_Labels = torch.zeros(self.Dataset_Size, dtype=torch.int64)
            dataset_index = 0

            # loop through each folder
            for Target, class_folder in enumerate(class_folders):
                
                class_folder_path = os.path.join(root_folder, class_folder)
                images_name = os.listdir(class_folder_path)
                images_name.sort()

                # loop through each sample
                for image_name in images_name:
                    
                    Total_Images = np.append(Total_Images, os.path.join(class_folder_path, image_name))
                    Total_Labels[dataset_index] = Target
                    dataset_index += 1

            permutation = np.random.permutation(np.arange(len(Total_Images)))
            
            self.images = Total_Images
            self.labels = Total_Labels

        #################### LOO Validation ####################

        elif Validation_Type == "LOO":

            if Train_Dataset:

                for class_folder in class_folders:
                    
                    class_validation_subjects = Validation_Subjects[class_folder]
                    class_folder_path = os.path.join(root_folder, class_folder)
                    subject_folders = os.listdir(class_folder_path)
                    subject_folders.sort()

                    for class_validation_subject in class_validation_subjects:
                        
                        if class_validation_subject[-1] != "*":

                            subject_folders.remove(class_validation_subject)


                    for subject_folder in subject_folders:

                        subject_folder_path = os.path.join(class_folder_path, subject_folder)
                        self.Dataset_Size += len(os.listdir(subject_folder_path))

                # define each image path and label
                Total_Images = np.empty(0)
                Total_Labels = torch.zeros(self.Dataset_Size, dtype=torch.int64)
                dataset_index = 0
                
                # looping through class folders
                for Target, class_folder in enumerate(class_folders):
                    
                    class_validation_subjects = Validation_Subjects[class_folder]
                    class_folder_path = os.path.join(root_folder, class_folder)

                    subject_folders = os.listdir(class_folder_path)
                    subject_folders.sort()

                    for class_validation_subject in class_validation_subjects:
                        
                        if class_validation_subject[-1] != "*":

                            subject_folders.remove(class_validation_subject)
                    

                    # looping through subjects
                    for subject_folder in subject_folders:

                        subject_folder_path = os.path.join(class_folder_path, subject_folder)
                        images_name = os.listdir(subject_folder_path)
                        images_name.sort()

                        # looping through images
                        for image_name in images_name:
                            # print(dataset_index)
                            Total_Images = np.append(Total_Images, os.path.join(subject_folder_path, image_name))
                            Total_Labels[dataset_index] = Target
                            dataset_index += 1

            elif not Train_Dataset:
                
                # looping through class folders
                for class_folder in class_folders:
                    
                    class_validation_Subjects = Validation_Subjects[class_folder]
                    class_folder_path = os.path.join(root_folder, class_folder)

                    subject_folders = class_validation_Subjects                      
                    subject_folders.sort()

                    for subject_folder in subject_folders:
                        
                        if subject_folder[-1] == "*":

                            subject_folder = subject_folder[0:-1]

                        subject_folder_path = os.path.join(class_folder_path, subject_folder)
                        self.Dataset_Size += len(os.listdir(subject_folder_path))

                Total_Images = np.empty(0)
                Total_Labels = torch.zeros(self.Dataset_Size, dtype=torch.int64)
                dataset_index = 0

                for Target, class_folder in enumerate(class_folders):

                    class_validation_subjects = Validation_Subjects[class_folder]
                    class_folder_path = os.path.join(root_folder, class_folder)

                    subject_folders = class_validation_subjects
                    subject_folders.sort()

                    for subject_folder in subject_folders:
                        
                        if subject_folder[-1] == "*":

                            subject_folder = subject_folder[0:-1]

                        subject_folder_path = os.path.join(class_folder_path, subject_folder)
                        images_name = os.listdir(subject_folder_path)
                        images_name.sort()

                        for image_name in images_name:

                            Total_Images = np.append(Total_Images, os.path.join(subject_folder_path, image_name))
                            Total_Labels[dataset_index] = Target
                            dataset_index += 1
            
            self.images = Total_Images
            self.labels = Total_Labels

    def __len__(self):

        return self.Dataset_Size

    def __getitem__(self, index):

        # Retriving Indexed Image
        image_path = self.images[index]
        image = imread(image_path)

        if self.XTransformer:
            
            image = self.XTransformer(image)

        # Retriving Indexed Label
        label = self.labels[index]

        if self.YTransformer:
            
            label = self.YTransformer(label)
        
        return image, label

## Network Class

In [6]:
class CNN_Model(nn.Module):
     
    def __init__(self, Input_Size, Number_of_Classes=2):
        
        super().__init__()
        Input_Channels = Input_Size[0]
        Temp_X = torch.reshape(torch.zeros(Input_Size), (1, *Input_Size))

        # Convolutional Layers
        self.Convolutional_Network = nn.Sequential(
            
            # First Layer
            nn.Conv2d(Input_Channels, 8, 3, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Second Layer
            nn.Conv2d(8, 16, 3, 1),
            nn.ReLU(inplace=True),
            # nn.Dropout(1),

            # Third Layer
            nn.Conv2d(16, 32, 3, 2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Fourth Layer
            nn.Conv2d(32, 96, 3, 2),
            nn.ReLU(inplace=True),
            nn.Flatten()
        )

        First_Linear_Layer_Size = self.Convolutional_Network(Temp_X).shape[1]

        # Linear Layers
        self.Linear_Network = nn.Sequential(

            # Fifth Layer
            nn.Linear(First_Linear_Layer_Size, 50),
            nn.ReLU(inplace=True),

            # Sixth Layer
            nn.Linear(50, 32),
            nn.ReLU(inplace=True),

            # Seventh Layer
            nn.Linear(32, Number_of_Classes),
            nn.Softmax(dim=1),
        )

    def forward(self, x):

        return self.Linear_Network(self.Convolutional_Network(x))

## Auxiliary Class

In [7]:
class ToNumpy():

    def __call__(self, input):

        return np.array(input)

#Functions

## Weight initializer

In [8]:
def weight_initializer(module):

    if isinstance(module, nn.Conv2d):
        nn.init.xavier_normal_(module.weight)
        nn.init.constant_(module.bias, 0.001)

    elif isinstance(module, nn.Linear):
        nn.init.xavier_normal_(module.weight)
        nn.init.constant_(module.bias, 0.001)

## Accuracy Computer

In [9]:
def Accuracy_Computer(Model, Dataset, Device):

    Corrects = 0
    Samples = 0
    Model.eval()

    with torch.no_grad():

        for image, label in Dataset:
            
            # Transfer Data to GPU
            image = image.to(device=Device)
            label = label.to(device=Device)

            # forward(prediction)
            label_prime = Model(image)
            _, predictions = label_prime.max(1)
            
            # determine the correct samples
            Corrects += (predictions == label).sum()
            Samples += label.shape[0]

        Accuracy =  (Corrects / Samples) * 100

    return float(Accuracy)

# Load Data

In [ ]:
# Validation_Type Options:

# 1-"KFold"
# 2-"Split"
# 3-"LOO"

Validation_Type = "LOO"

if Validation_Type == "KFold":

    Root_Folder = "AllSubjects"

elif Validation_Type == "Split":

    Root_Folder = "AllSubjects"

elif Validation_Type == "LOO":

    Root_Folder = "SubjectWise"

else:

    raise ValueError("no option is choosed")

if not os.path.isdir(f"/content/{Root_Folder}"):

    if not os.path.isdir("/content/Drive"):

        drive.mount("Drive", force_remount=True)

    ZipFile = f'/content/Drive/MyDrive/Thesis/{Root_Folder}.zip'
    !unzip -qq {ZipFile}

# Hyper Parameters

In [ ]:
# Dataset Parameters
Batch_Size = 150
Train_Split = 0.9
Composit_Transformer = transforms.Compose([ToNumpy(), transforms.ToTensor()])

# Training Parameters
Folds = 10
Epochs = 30
Learning_Rate = 1e-4
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset

In [ ]:
if Validation_Type == "KFold":

    Full_Dataset = Medical_Dataset(f"/content/{Root_Folder}", XTransformer=Composit_Transformer)
    image, _ = Full_Dataset.__getitem__(0)

    if Folds > 1 :

        kfold = KFold(n_splits=Folds, shuffle=True)
        Data_Fold = kfold.split(Full_Dataset)

    else:

        raise ValueError("Number of Folds should be greather than 1 when validation type is set to KFold")

    Train_Size = round((Folds - 1)/Folds * len(Full_Dataset))
    Test_Size = round(1/Folds * len(Full_Dataset))

elif Validation_Type == "Split":

    Folds = 1
    Full_Dataset = Medical_Dataset(f"/content/{Root_Folder}", XTransformer=Composit_Transformer)
    image, _ = Full_Dataset.__getitem__(0)

    Train_Size = int(len(Full_Dataset) * Train_Split)
    Test_Size = len(Full_Dataset) - Train_Size

    Train_Dataset, Test_Dataset = random_split(Full_Dataset, [Train_Size, Test_Size])

elif Validation_Type == "LOO":
    
    Folds = 1
    Validation_Subjects = {"HC":[], "PD":["S_13"]}

    Train_Dataset = Medical_Dataset(f"/content/{Root_Folder}", XTransformer=Composit_Transformer, Validation_Type=Validation_Type, 
                                    Train_Dataset=True, Validation_Subjects=Validation_Subjects)
    Test_Dataset = Medical_Dataset(f"/content/{Root_Folder}", XTransformer=Composit_Transformer, Validation_Type=Validation_Type, 
                                   Train_Dataset=False, Validation_Subjects=Validation_Subjects)
    
    Train_Size = len(Train_Dataset)
    Test_Size = len(Test_Dataset)

    image, _ = Train_Dataset.__getitem__(0)

input_size = image.shape

print(f"Train Size: {Train_Size}")
print(f"Test Size: {Test_Size}")

# Defining and Initializing the Model

In [ ]:
# Defining the Model
Model = CNN_Model(input_size, 2).to(Device)

# Summary of the Model
Summary(Model, input_size=input_size)

# Defining Loss Function and Optimizer

In [ ]:
# Defining Loss Function
Criteria = nn.CrossEntropyLoss()

# Defining Optimizer
Optimizer = optim.Adam(Model.parameters(), lr=Learning_Rate, weight_decay=1e-2)

# Training the Model

In [ ]:
Total_Train_Accuracy = np.zeros((Epochs, Folds))
Total_Test_Accuracy = np.zeros((Epochs, Folds))

Total_Train_Loss = np.zeros((Epochs, Folds))
Total_Test_Loss = np.zeros((Epochs, Folds))

for fold in range(1, Folds + 1):

    if Validation_Type == "KFold":

        Train_indices, Test_indices = next(iter(Data_Fold))

        Train_Sampler = SubsetRandomSampler(Train_indices)
        Test_Sampler = SubsetRandomSampler(Test_indices)
        Test_Size = len(Test_indices)

        Train_Loader = DataLoader(dataset=Full_Dataset, batch_size=Batch_Size, sampler=Train_Sampler)
        Test_Loader = DataLoader(dataset=Full_Dataset, batch_size=Test_Size, sampler=Test_Sampler)

        print("\n",
        f"Fold {fold}/{Folds}",
        "\n",
        "=" * 25,
        "\n", sep="")
    
    elif Validation_Type == "Split":

        Train_Loader = DataLoader(dataset=Train_Dataset, batch_size=Batch_Size, shuffle=True, pin_memory=True, num_workers=2)
        Test_Loader = DataLoader(dataset=Test_Dataset, batch_size=Test_Size, shuffle=True, pin_memory=True, num_workers=2)

    elif Validation_Type == "LOO":

        Train_Loader = DataLoader(dataset=Train_Dataset, batch_size=Batch_Size, shuffle=True, pin_memory=True, num_workers=2)
        Test_Loader = DataLoader(dataset=Test_Dataset, batch_size=Test_Size, shuffle=True, pin_memory=True, num_workers=2)

    # Initializing the Model
    Model.apply(weight_initializer)

    # Loop Through Epochs
    for epoch in range(1, Epochs + 1):

            Start = time()
            Train_Loss = 0
            Test_Loss = 0

            # Loop Through Bacthes
            for Batch_Number, (image, label) in enumerate(Train_Loader, 1):
                
                # Retrive the batch of image and labels from Dataset
                image = image.to(device=Device)
                label = label.to(device=Device)
                
                # zero the parameter gradient
                Optimizer.zero_grad()

                # Forward Step
                output = Model(image)
                
                # Backward Step
                Loss = Criteria(output, label)
                Loss.backward()

                # Opimization Step
                Optimizer.step()

                # Calculating the Loss in an epoch
                Train_Loss += Loss.item()
            
            Train_Loss = Train_Loss / Batch_Number

            # Calculating the Test Loss
            with torch.no_grad():

                image , label = next(iter(Test_Loader))

                # # Transfer data to Device
                label = label.to(Device)
                image = image.to(Device)

                # Forward Propagation and Calculate the Loss
                output = Model(image)
                Loss = Criteria(output, label)

                # Calculating loss for all the Test Dataset
                Test_Loss += Loss.item()

            Train_Accuracy =  Accuracy_Computer(Model, Train_Loader, Device)
            Test_Accuracy = Accuracy_Computer(Model, Test_Loader, Device)

            Total_Train_Loss[epoch - 1, fold - 1] = Train_Loss
            Total_Test_Loss[epoch - 1, fold - 1] = Test_Loss

            Total_Train_Accuracy[epoch - 1, fold - 1] = Train_Accuracy
            Total_Test_Accuracy[epoch - 1, fold - 1] = Test_Accuracy

            End = time()
            print(f"Epoch {epoch}/{Epochs} : (Elapsed Time = {round(End-Start, 2)} Seconds)\n {' ' * 5} "
                  f"[Train Loss = {Train_Loss:.5f}] - [Train Accuracy = {(Train_Accuracy):.2f}%] | "
                  f"[Test Loss = {Test_Loss:.5f}] - [Test Accuracy = {(Test_Accuracy):.2f}%] \n "
                  f"{' ' * 4 + '-' * 105}")

In [ ]:
for fold in range(1 , Folds + 1):
    
    if Validation_Type == "KFold":

        print("\n",
              f"Fold {fold}/{Folds}",
              "\n",
              "=" * 25,
              "\n", sep="")

    plt.subplot(1, 2, 1)
    plt.plot(Total_Train_Accuracy[:, fold - 1])
    plt.plot(Total_Test_Accuracy [:, fold - 1])
    plt.legend(["Train Accuracy", "Test Accuracy"])

    plt.subplot(1, 2, 2)
    plt.plot(Total_Train_Loss[:, fold - 1])
    plt.plot(Total_Test_Loss[:, fold - 1])
    plt.legend(["Train Loss", "Test Loss"])

    plt.show()